In [1]:
import requests
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
#creating an model object
model = ChatOpenAI()
model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000180892C7400>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001808A79FB80>, root_client=<openai.OpenAI object at 0x00000180892C5240>, root_async_client=<openai.AsyncOpenAI object at 0x000001808A79FAF0>, model_kwargs={}, openai_api_key=SecretStr('**********'))

# first creating Custom Tools

# step1:- created custom tool

In [3]:
from langchain_core.tools import tool
#tools in langchain is nothing but user defined function.

@tool
def multiplication(a:int,b:int) ->int:
    """Given 2 number a and b this tool will returns their product"""
    return a*b

# step:2 Binding the tool with llm

# now binding the tool with llm

In [ ]:
#wrapping the tools with LLM
bind_tool = model.bind_tools([multiplication])
bind_tool

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000180892C7400>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001808A79FB80>, root_client=<openai.OpenAI object at 0x00000180892C5240>, root_async_client=<openai.AsyncOpenAI object at 0x000001808A79FAF0>, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiplication', 'description': 'Given 2 number a and b this tool will returns their product', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]}, config={}, config_factories=[])

# Impt step:- Here we gonna discuss how llm model use specific tool or call tool based on the task

# TOOL CALLING

In [ ]:
#if i ask generally query to llm it will respond sam manner like below example
bind_tool.invoke("hi how are you")

AIMessage(content="Hello! I'm here and ready to assist you. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 59, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CAFrzKpayHvHgYf6Utlx4WRql2GOX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d8e17064-7181-424f-a6c0-ea0fa1c28f86-0', usage_metadata={'input_tokens': 59, 'output_tokens': 19, 'total_tokens': 78, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
#case:-2 
bind_tool.invoke("can you multiply 3 with 2") #in this response you are getting empty content 

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7sGx6zpHt4E9rXZj5NgSYGvz', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiplication'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 63, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CAFtiP7sXhl31346pqNRpRn0NWR95', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b20b0636-b9a3-4126-afcf-76d21167caa8-0', tool_calls=[{'name': 'multiplication', 'args': {'a': 3, 'b': 2}, 'id': 'call_7sGx6zpHt4E9rXZj5NgSYGvz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 19, 'total_tokens': 82, 'input_token_details': {'audio': 0, 'cache_r

```
Note:- the llm doesnt actually run the tools--->it just suggests the tool(given query ke liye konsa tool sahi rahegha) and the input arguments.
the actually execution is handled by you or Langchain
```

In [7]:
#we can fetch the tool call attributes
bind_tool.invoke("can you multiply 3 with 2").tool_calls

[{'name': 'multiplication',
  'args': {'a': 3, 'b': 2},
  'id': 'call_RQKyHjljHEq9Zx73TwZS96q4',
  'type': 'tool_call'}]

In [8]:
#fetching 0th position tool call from lst object

bind_tool.invoke("can you multiply 3 with 2").tool_calls[0]

{'name': 'multiplication',
 'args': {'a': 3, 'b': 2},
 'id': 'call_65CYMqCXhM2obFjT5ZFq2EQN',
 'type': 'tool_call'}

# step:3 Exceuting the tools

```
tools execution means :- actual python function(tool that created user defined func ki help se) is run by using the INPUT ARGUMENT suggested by LLM during tool calling

```

In [ ]:
#bind the tool with llm
result = bind_tool.invoke("can you multiply 3 with 2")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Mh76QowpjYoBezAdBfoXoZuJ', 'function': {'arguments': '{"a":3,"b":2}', 'name': 'multiplication'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 63, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CAG7vLwEyIVgOlrEnfy7lsyjy5cze', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8a3287a5-c5bd-4b13-be85-7bcb10f104b3-0', tool_calls=[{'name': 'multiplication', 'args': {'a': 3, 'b': 2}, 'id': 'call_Mh76QowpjYoBezAdBfoXoZuJ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 19, 'total_tokens': 82, 'input_token_details': {'audio': 0, 'cache_r

In [11]:
result.tool_calls[0]

{'name': 'multiplication',
 'args': {'a': 3, 'b': 2},
 'id': 'call_Mh76QowpjYoBezAdBfoXoZuJ',
 'type': 'tool_call'}

In [12]:
#now executing the tools
multiplication.invoke(input=result.tool_calls[0]['args'])

6

In [ ]:
multiplication.invoke(input=result.tool_calls[0]) #tool msg we get only whenever any tool executed by model.

ToolMessage(content='6', name='multiplication', tool_call_id='call_Mh76QowpjYoBezAdBfoXoZuJ')